In [0]:
%run "./000_setup_storage"

In [0]:
bronze_table_path = bronze_path + "orders/"
bronze_orders = spark.read.format("delta").load(bronze_table_path)


In [0]:
# # Databricks notebook source
# # -----------------------------
# # STEP 0: Paths
# silver_path = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/silver/orders/"

# # Read Bronze table
# bronze_orders = spark.read.table("ws_databricks_etl.bronze.orders")
# display(bronze_orders)


In [0]:
# Databricks notebook source
# -----------------------------
# STEP 1: Apply same business rules for Silver
from pyspark.sql.functions import upper
from pyspark.sql.functions import col, lit, when, expr
from pyspark.sql.functions import current_timestamp



silver_orders = bronze_orders.filter(
    (col("order_id").isNotNull()) &
    (col("status").isNotNull()) &
    ((col("amount").isNotNull()) | (col("status") == "cancelled"))
)

# Standardize customer names
if "customer_name" in silver_orders.columns:
    silver_orders = silver_orders.withColumn("customer_name", upper(col("customer_name")))

# Add transformation timestamp
silver_orders = silver_orders.withColumn("transformation_timestamp", current_timestamp())

display(silver_orders)


In [0]:
# Databricks notebook source
# -----------------------------
# STEP 2: Write Silver Delta
silver_orders_path = f"{silver_path}orders"

silver_orders.write.format("delta").mode("overwrite").save(silver_orders_path)

spark.sql(f"""
CREATE TABLE IF NOT EXISTS ws_databricks_etl.silver.orders
USING DELTA
LOCATION '{silver_orders_path}'
""")

In [0]:
%sql
select * from ws_databricks_etl.silver.orders